In [ ]:
# 필요한 라이브러리를 설치합니다.
# KoNLPy의 Mecab 형태소 분석기는 외부로부터 추가 설치가 필요하기 때문에 pip에서 배포하는 패키지 설치 시간보다 조금 더 오래 걸립니다.

!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

!pip install transformers datasets wandb

+ pip install konlpy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
+ bash -x
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ hash mecab
+ echo 'mecab-ko is already installed'
mecab-ko is already installed
+ [[ -d /usr/local/lib/mecab/dic/mecab-ko-dic ]]
+ echo 'mecab-ko-dic is already installed'
mecab-ko-dic is already installed
++ python3 -c 'import pkgutil; print(1 if pkgutil.find_loader("MeCab") else 0)'
+ [[ 1 == \1 ]]
+ echo 'mecab-python is already installed'
mecab-python is already installed
+ echo Done.
Done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
from google.colab import drive

drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive/WandB_Sweep_test")

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

fine_tuned_model_name = "Context_Sampling_Jungmin_v1.4"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!wandb login

import wandb

wandb: Currently logged in as: jmp0813 (2nd_group). Use `wandb login --relogin` to force relogin


In [ ]:
import pandas as pd
import json
import torch
import datasets
import numpy as np
import random
import nltk
import re
import torch.nn.functional as F
import utils

from glob import glob
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, DataCollatorWithPadding, get_cosine_schedule_with_warmup
from transformers import AutoModelForQuestionAnswering
from collections import defaultdict, deque
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

SEED = 20220803

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if "cuda" in DEVICE.type :
    torch.cuda.set_device(DEVICE)
print(DEVICE)

tagger = Mecab()

cuda:0


In [ ]:
# Sweep을 돌리기 위해 저희가 가지고 있었던 모든 내용들을 전부 메소드로 만듭니다.
# 훈련 루프를 포함해 데이터를 가져오는 것 부터 전부 메소드로 만들었습니다.

In [ ]:
# 탐색할 하이퍼 파라미터 또는 인자들에 대한 정의입니다.
# sweep은 아래 인자들을 돌아다니며 여러 모델을 학습시킵니다.

sweep_config = {
    "name" : "bert_sweep", # sweep에 출력될 이름입니다.
        'metric' : {
        'name': 'loss',
        'goal': 'minimize'   
        },
    "method" : "random",    # 탐색 방법입니다. grid나 bayes 등의 옵션도 존재합니다.
    "parameters" : {
        'optimizer': {
            'values': ['AdamW', 'RMSprop']  # 사용할 옵티마이저의 종류입니다. 다른 옵티마이저도 사용 가능한지 임의로 RMSProp을 넣어뒀습니다. 다른 옵티마이저를 추가하실 분은 아래 sweep_optimizer 함수에 추가하시고 사용하시면 됩니다. 
        },
        "backbone" : {
            "values" : ["kykim/bert-kor-base", "kykim/electra-kor-base"] # 사용할 사전학습 모델 이름들입니다.
        },
        "epochs" : {
        "values" : [3, 5]               # 에포크 크기입니다.
        },
        "use_lr_scheduler" : {
            "values" : [True, False]        # 학습률 스케줄링을 할지 말지 결정합니다.
        },
        "start_logits_loss_weight" : {      # 토큰의 종료 인덱스를 잘 맞추지 못해서 가중 손실함수를 쓰자는 아이디어가 있었습니다. 가중치 탐색을 위한 인자입니다.
            "min" : 0.1,
            "max" : 0.5
        },
        "lr_scheduler_warmup_ratio" : {     # 만일 학습률 스케줄링을 사용한다면 전체 training step 중 몇 번에 걸쳐 최고 learning rate에 도달할지 결정하는 인자입니다.
            "min" : 0.02,
            "max" : 0.4
        },
        "learning_rate" : {
            "min": 1e-6,
            "max": 1e-4
        },
        "batch_size" : {    # 배치 크기입니다. 16이 넘어가면 OOM이 뜨니 참고해주세요!
            'values' : [8, 16]
        }
    }
}


In [ ]:
def run_sweep(config = None):
    """
    sweep_config로부터 얻은 parameter를 이용해 학습하는 함수입니다.
    사실상 저희가 그동안 사용했던 데이터 불러오기~학습까지 모든 과정이 담긴 함수입니다.    
    사전학습 모델마다 tokenizer가 다르기 때문에 sweep이 바뀔때마다 데이터 또한 새로 샘플링해줘야 합니다. (padding value를 추출해야 하기 때문에 collator가 tokenizer를 인자로 받습니다.)
    따라서 sweep을 한 번 돌 때 마다 baseline.ipynb 파일을 새로 실행한다고 보시면 됩니다.
    """

    with wandb.init(config=config) :
        w_config = wandb.config

        # 지금 sweep에서 사용할 사전학습 모델과 tokenizer를 불러옵니다.
        backbone = w_config.backbone
        tokenizer = AutoTokenizer.from_pretrained(backbone, do_lower_case = False)
        collator = DataCollatorWithPadding(tokenizer, return_tensors = "pt")
        model = create_model(backbone).to(DEVICE)


        # 지금 sweep에서 사용할 데이터를 불러옵니다.
        original_train = utils.load_data("./RawData/train.json")
        original_train, rows_to_drop = utils.sampling_context_with_cosine_similarity(original_train, tokenizer, tagger)
        original_train.loc[rows_to_drop, "answer_start"] = 0
        original_train.loc[rows_to_drop, "answer_end"] = 0
        original_train, original_valid = train_test_split(original_train, test_size = 0.3, random_state = SEED)
        train_data, valid_data = sweep_dataset(original_train, original_valid, tokenizer, collator, w_config.batch_size)

        # 지금 sweep에서 사용할 optimizer와 learning rate scheduler를 불러옵니다.
        # 위 config를 수정하지 않으셨다면 경우에 따라 learning rate scheduler를 사용하지 않을 수 있습니다. 따라서 사용하지 않을 경우 None이 저장됩니다.
        optimizer = sweep_optimizer(model, w_config.optimizer, w_config.learning_rate)
        lr_scheduler = sweep_lr_scheduler(optimizer, len(train_data) * w_config.epochs, w_config.lr_scheduler_warmup_ratio) if w_config.use_lr_scheduler else None

        # epoch만큼 훈련 루프와 검증 루프를 돌립니다.
        for epoch in range(w_config.epochs):
            train_loss, train_dist, train_start_acc, train_end_acc = train_epoch(train_data, model, epoch, optimizer, DEVICE, w_config.start_logits_loss_weight, tokenizer, tagger, lr_scheduler)
            valid_loss, valid_dist, valid_start_acc, valid_end_acc = valid_epoch(valid_data, model, epoch, DEVICE, w_config.start_logits_loss_weight, tokenizer)
            wandb.log({"distance": valid_dist,
                       "loss" : valid_loss})

In [1]:
def sweep_dataset(train, valid, tokenizer, collator, batch_size) :
    """
    현재 sweep에서 사용할 훈련 / 검증 데이터를 추출합니다.

    Argument
    train : json으로부터 parsing이 완료된 pandas Dataframe 중 훈련용으로 구분된 데이터를 받습니다.
    valid : json으로부터 parsing이 완료된 pandas Dataframe 중 검증용으로 구분된 데이터를 받습니다.
    tokenizer : 현재 sweep에서 사용하는 사전학습 모델의 tokenizer를 받습니다.
    batch_size : config에 의해 결정된 현재 sweep의 batch_size를 받습니다.
    """
    return utils.get_dataset(train, tokenizer, collator, batch_size, True), utils.get_dataset(valid, tokenizer, collator, batch_size * 2, True)

def sweep_optimizer(input_model, optimizer, learning_rate) :
    """
    현재 sweep에서 사용할 optimizer를 반환합니다.

    Argument
    input_model : 사전학습 모델을 입력받습니다. optimizer와 weight를 연결하기 위해 입력받습니다.
    optimiezr : config에 의해 결정된 optimizer의 이름을 문자열로 입력받습니다.
    learning_rate : config에 의해 결정된 learning_rate를 입력받습니다.
    """
    if optimizer == "AdamW" :
        optimizer = torch.optim.AdamW(input_model.parameters(), lr = learning_rate, eps = 1e-6, weight_decay = 0.02)
    elif optimizer == 'RMSprop' :
        optimizer = torch.optim.RMSprop(input_model.parameters(), lr = learning_rate, weight_decay = 0.02)    
    return optimizer

def sweep_lr_scheduler(optimizer, train_step, warmup_ratio) :
    """
    만일 현재 sweep에서 learning rate scheduler를 사용할 경우 이를 정의하고 반환합니다.

    Argument
    optimizer : 위 sweep_optimizer로부터 얻은 optimizer를 입력받습니다.
    train_step : 전체 훈련 스텝을 입력받습니다. 원본 데이터 크기 / batch_size * epochs가 전체 훈련 스텝 크기입니다.
    warmup_ratio : 전체 훈련 스텝 중 어느 시점까지 learning rate를 증가시킬 것인지 결정합니다.
    """
    lr_scheduler = get_cosine_schedule_with_warmup(optimizer = optimizer,
                                                    num_warmup_steps = int(train_step * warmup_ratio),
                                                    num_training_steps = train_step)
    return lr_scheduler


def create_model(backbone) :
    model = AutoModelForQuestionAnswering.from_pretrained(backbone)
    model.train()
    return model

In [ ]:
def train_epoch(train_data, model, epoch, optimizer, device, start_weight, tokenizer, tagger, lr_scheduler = None) :
    """
    1번의 training epoch를 정의한 함수입니다.
    해당 함수는 sweep에 의해 결정된 epoch 횟수만큼 호출됩니다.
    훈련 루프를 함수로 만들었기 때문에 훈련 진행 간 바뀔 수 있는 객체를 인자로 넣어 사용합니다.
    """

    scaler = torch.cuda.amp.GradScaler()

    cum_loss = deque(maxlen = 20)
    cum_dist = deque(maxlen = 20)
    cum_start_acc = deque(maxlen = 20)
    cum_end_acc = deque(maxlen = 20)

    curr_loss = []
    curr_dist = []
    curr_start_acc = []
    curr_end_acc = []

    model.train()

    with tqdm(train_data, unit = " batch") as tepoch :
        curr_loss.clear()
        model.train()
        for i, batch in enumerate(tepoch) :
            optimizer.zero_grad()

            tepoch.set_description(f"Train Epoch {epoch}")
            batch = {k : v.to(device) for k, v in batch.items()}
            
            with torch.cuda.amp.autocast() :
                outputs = model(**batch)
                start_logits = outputs["start_logits"]
                end_logits = outputs["end_logits"]
                loss = utils.weighted_loss_fn(start_logits, end_logits, batch["start_positions"], batch["end_positions"], start_weight, 1 - start_weight)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            if lr_scheduler :
                lr_scheduler.step()

            cum_loss.append(float(loss))
            curr_loss.append(float(loss))

            start_acc, end_acc = utils.extract_accuracy(start_logits, end_logits, batch["start_positions"], batch["end_positions"])
            cum_start_acc.append(float(start_acc))
            cum_end_acc.append(float(end_acc))
            curr_start_acc.append(float(start_acc))
            curr_end_acc.append(float(end_acc))

            dist = utils.levenshtein_distance(start_logits, end_logits, batch["start_positions"], batch["end_positions"], batch["input_ids"], tokenizer, tagger)
            cum_dist.append(float(dist))
            curr_dist.append(float(dist))

            tepoch.set_postfix(loss = sum(cum_loss) / len(cum_loss),
                               dist = sum(cum_dist) / len(cum_dist),
                               start_acc = sum(cum_start_acc) / len(cum_start_acc),
                               end_acc = sum(cum_end_acc) / len(cum_end_acc))
            
            wandb.log({"train_loss" : sum(cum_loss) / len(cum_loss),
                       "train_dist" : sum(cum_dist) / len(cum_dist),
                       "train_start_acc" : sum(cum_start_acc) / len(cum_start_acc),
                       "train_end_acc" : sum(cum_end_acc) / len(cum_end_acc),
                       "lr" : optimizer.state_dict()["param_groups"][0]['lr'],
                       "train_step" : i + (len(train_data) * epoch)})
            
        print("Train loss : ", sum(curr_loss) / len(curr_loss))
        print("Train dist : ", sum(curr_dist) / len(curr_dist))
        print("Train start acc :", sum(curr_start_acc) / len(curr_start_acc))
        print("Train end acc :", sum(curr_end_acc) / len(curr_end_acc))
            
    return sum(curr_loss) / len(curr_loss), sum(curr_dist) / len(curr_dist), sum(curr_start_acc) / len(curr_start_acc), sum(curr_end_acc) / len(curr_end_acc)

def valid_epoch(valid_data, model, epoch, device, start_weight, tokenizer) :
    """
    1번의 validation epoch를 정의한 함수입니다.
    해당 함수는 sweep에 의해 결정된 epoch 횟수만큼 호출됩니다.
    검증 루프를 함수로 만들었기 때문에 검증 진행 간 바뀔 수 있는 객체를 인자로 넣어 사용합니다.
    """

    cum_loss = deque(maxlen = 20)
    cum_dist = deque(maxlen = 20)
    cum_start_acc = deque(maxlen = 20)
    cum_end_acc = deque(maxlen = 20)

    curr_loss = []
    curr_dist = []
    curr_start_acc = []
    curr_end_acc = []

    model.eval()
    with torch.no_grad() :
        with tqdm(valid_data, unit = " batch") as tepoch :
            model.eval()
            with torch.no_grad() :
                for i, batch in enumerate(tepoch) :
                    tepoch.set_description(f"Valid Epoch {epoch}")
                    batch = {k : v.to(device) for k, v in batch.items()}
                    
                    with torch.cuda.amp.autocast() :
                        outputs = model(**batch)
                        start_logits = outputs["start_logits"]
                        end_logits = outputs["end_logits"]
                        loss = utils.weighted_loss_fn(start_logits, end_logits, batch["start_positions"], batch["end_positions"], start_weight, 1 - start_weight)


                    cum_loss.append(float(loss))
                    curr_loss.append(float(loss))

                    start_acc, end_acc = utils.extract_accuracy(start_logits, end_logits, batch["start_positions"], batch["end_positions"])
                    cum_start_acc.append(float(start_acc))
                    cum_end_acc.append(float(end_acc))
                    curr_start_acc.append(float(start_acc))
                    curr_end_acc.append(float(end_acc))

                    dist = utils.levenshtein_distance(start_logits, end_logits, batch["start_positions"], batch["end_positions"], batch["input_ids"], tokenizer, tagger)
                    cum_dist.append(float(dist))
                    curr_dist.append(float(dist))

                    tepoch.set_postfix(loss = sum(cum_loss) / len(cum_loss),
                                    dist = sum(cum_dist) / len(cum_dist),
                                start_acc = sum(cum_start_acc) / len(cum_start_acc),
                                end_acc = sum(cum_end_acc) / len(cum_end_acc))
                    
                    wandb.log({"valid_loss" : sum(cum_loss) / len(cum_loss),
                               "valid_dist" : sum(cum_dist) / len(cum_dist),
                               "valid_start_acc" : sum(cum_start_acc) / len(cum_start_acc),
                               "valid_end_acc" : sum(cum_end_acc) / len(cum_end_acc),
                               "valid_step" : i + (len(valid_data) * epoch)})
                    
        print("Valid loss : ", sum(curr_loss) / len(curr_loss))
        print("Valid dist : ", sum(curr_dist) / len(curr_dist))
        print("Valid start acc :", sum(curr_start_acc) / len(curr_start_acc))
        print("Valid end acc :", sum(curr_end_acc) / len(curr_end_acc))
                
    return sum(curr_loss) / len(curr_loss), sum(curr_dist) / len(curr_dist), sum(curr_start_acc) / len(curr_start_acc), sum(curr_end_acc) / len(curr_end_acc)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project = "Goorm_2nd_project", entity = "2nd_group")

Create sweep with ID: sjwplq4l
Sweep URL: https://wandb.ai/2nd_group/Goorm_2nd_project/sweeps/sjwplq4l


In [ ]:
wandb.agent(sweep_id, run_sweep, count = 10) # count는 sweep을 실행할 횟수입니다. baseline.ipynb 파일 전체를 실행하는데 약 1시간이 걸렸을 때 count가 10이라면 이 sweep은 10시간동안 돌아갑니다. 참고하셔서 실행해주세요!

wandb: Agent Starting Run: ch4gha3i with config:
wandb: 	backbone: kykim/electra-kor-base
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	learning_rate: 9.207582227919752e-05
wandb: 	lr_scheduler_warmup_ratio: 0.047735790354572816
wandb: 	optimizer: AdamW
wandb: 	start_logits_loss_weight: 0.49984319581306424
wandb: 	use_lr_scheduler: True
wandb: Currently logged in as: jmp0813 (2nd_group). Use `wandb login --relogin` to force relogin


100%|██████████| 9789/9789 [00:01<00:00, 5193.54it/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (741 > 512). Running this sequence through the model will result in indexing errors


코드 오류로 추정되는 문제로 인해 사용하지 못하는 문장의 비율 : 0.000551808706315144
유사도 기반으로 추출한 문장에 정답 단어가 없는 문장의 비율 : 0.019558553034947884
위 두 문제를 합한 전체 오류 : 0.02011036174126303
실제로 유사도 기반 추출을 한 문장들 중 오류가 있는 문장의 비율: 0.07677902621722846
전체 문장들 중 사용할 수 있는 문장의 비율 : 0.9799509503372165


  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

...

Train Epoch 1: 100%|██████████| 1428/1428 [10:07<00:00,  2.35 batch/s, dist=26.6, end_acc=0.525, loss=1.42, start_acc=0.806]


Train loss :  1.1654455481457109
Train dist :  20.20045518207283
Train start acc : 0.8015581232492998
Train end acc : 0.6425070028011205


Valid Epoch 1: 100%|██████████| 306/306 [01:32<00:00,  3.29 batch/s, dist=12.6, end_acc=0.651, loss=1.28, start_acc=0.803]


Valid loss :  1.3042493678775489
Valid dist :  18.247501885369534
Valid start acc : 0.799648064413881
Valid end acc : 0.6138448970380173


Train Epoch 2:  90%|████████▉ | 1281/1428 [09:03<01:12,  2.02 batch/s, dist=24.3, end_acc=0.55, loss=1.64, start_acc=0.675]